In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup as bs
import requests

In [2]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

In [3]:
import json
import tweepy
from config import consumer_key, consumer_secret, access_token, access_token_secret

### NASA Mars News

In [4]:
!which chromedriver

/usr/local/bin/chromedriver


In [5]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)

In [7]:
news_html = browser.html
news_soup = bs(news_html, 'html.parser')
#print(news_soup.prettify())

In [8]:
items = news_soup.find_all('ul',class_='item_list')
for item in items:
    news_title = item.find('div',class_='content_title').text
    news_p = item.find('div', class_='article_teaser_body').text
    print("----------------")
    print("Latest News")
    print(f"Title: "+news_title)
    print(news_p)
    


----------------
Latest News
Title: Third ASPIRE Test Confirms Mars 2020 Parachute a Go
The supersonic parachute that will handle the heaviest payload yet to the Red Planet – Mars 2020 rover – passes its final sounding rocket test with flying colors.


### JPL Mars Space Images

In [9]:
!which chromedriver

/usr/local/bin/chromedriver


In [10]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [11]:
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [12]:
img_html = browser.html
img_soup = bs(img_html, 'html.parser')
#print(img_soup.prettify())

In [13]:
#featured = img_soup.find_all(
feat = img_soup.find('footer')
data_link = feat.find('a')['data-link']
data_img = ('https://www.jpl.nasa.gov/' + data_link)

browser.visit(data_img)
feat_img_html = browser.html
feat_img_soup = bs(feat_img_html, 'html.parser')
feat_img = feat_img_soup.find('figure', class_='lede')
feat_img_href = feat_img.find('a')['href']


In [14]:
featured_image_url = (f'https://www.jpl.nasa.gov/' + feat_img_href)

In [15]:
featured_image_url

'https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA17357_hires.jpg'

### Mars Weather

In [16]:
# Setup Tweepy API authentications
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [17]:
# target user for Mars Weather
target_user = "MarsWxReport"

In [18]:
# get tweets from target
tweets = api.user_timeline(target_user, page=1, result_type="recent")

In [19]:
mars_weather = tweets[0]['text']
mars_weather

'Sol 2209 (2018-10-23), high -18C/0F, low -73C/-99F, pressure at 8.79 hPa, daylight 06:09-18:26'

### Mars Facts

In [20]:
facts_url = "https://space-facts.com/mars/"

In [21]:
table = pd.read_html(facts_url)

mars_table_to_html =table[0].to_html(header=False, index=False)
mars_table_to_html


'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

### Mars Hemispheres

In [22]:
!which chromedriver

/usr/local/bin/chromedriver


In [23]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [24]:
hem_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hem_url)

In [25]:
link_list = []
url_list = []


hem_html = browser.html
hem_soup = bs(hem_html, 'html.parser')
results = hem_soup.find_all('div',class_='item')
for result in results:
    link = result.find('a')['href']
    link_list.append(link)
    
url_list = ['https://astrogeology.usgs.gov/'+ link for link in link_list]

In [26]:
hemisphere_images = []

for page in url_list:      
    page_url = page
    browser.visit(page_url)
    page_html = browser.html
    page_soup = bs(page_html, 'html.parser')
        
    title = page_soup.find('h2',class_='title').text
  
    image = page_soup.find('img', class_='wide-image')
    img_url = (f"https://astrogeology.usgs.gov" + image["src"])

    hem_dict = {
        'title': title,
        'img_url': img_url
    }
    hemisphere_images.append(hem_dict)



In [27]:
hemisphere_images

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [28]:
!jupyter nbconvert --to script mission_to_mars.ipynb

[NbConvertApp] Converting notebook mission_to_mars.ipynb to script
[NbConvertApp] Writing 3970 bytes to mission_to_mars.py
